In [3]:
import numpy as np
import pandas as pd
from config import *

In [4]:
def filter_interaction(df_inter):
    print('origin shape: ', df_inter.shape)
    vc_user = df_inter['user_id'].value_counts()
    ls_user = vc_user[vc_user >= 3].index
    mask = df_inter['user_id'].isin(ls_user)
    df_inter = df_inter.loc[mask].reset_index(drop=True)
    print('> 3 interactions: ', df_inter.shape)
    return df_inter, ls_user

## douban book

In [5]:
df_douban_inter = pd.read_csv(data_path_douban + 'user_book.dat', sep='\t', header=None).rename({
    0: 'user_id',
    1: 'item_id',
    2: 'rating',
}, axis=1)
df_douban_inter = df_douban_inter[['user_id', 'item_id', 'rating']]
df_douban_inter['user_id'] = df_douban_inter['user_id'].astype('str')
df_douban_inter['item_id'] = df_douban_inter['item_id'].astype('str')
df_douban_inter, ls_user_douban = filter_interaction(df_douban_inter)

df_douban_inter.to_csv(data_preprocessing + "douban_inter.csv", index=False)

origin shape:  (792062, 3)
> 3 interactions:  (790197, 3)


In [6]:
# multihot
df_douban_location = pd.read_csv(data_path_douban + 'user_location.dat', sep='\t', header=None)
df_douban_group = pd.read_csv(data_path_douban + 'user_group.dat', sep='\t', header=None)

In [ ]:
df_user_user = pd.read_csv(data_path_douban + 'user_user.dat', sep='\t', header=None)

,0,1
0,5479,6650
1,5800,7722
2,6951,1009
3,5479,6656
4,5355,1759
...,...,...
169145,9080,8694
169146,10595,2736
169147,10315,3837
169148,5321,11715


## movie lens

In [7]:
df_movie_inter = pd.read_csv(data_path_movie + 'user_movie.dat', sep='\t', header=None).rename({
    0: 'user_id',
    1: 'item_id',
    2: 'rating',
}, axis=1)
df_movie_inter = df_movie_inter[['user_id', 'item_id', 'rating']]
df_movie_inter['user_id'] = df_movie_inter['user_id'].astype('str')
df_movie_inter['item_id'] = df_movie_inter['item_id'].astype('str')
df_movie_inter, ls_user_movie  = filter_interaction(df_movie_inter)

df_movie_inter.to_csv(data_preprocessing + "movie_inter.csv", index=False)

origin shape:  (100000, 3)
> 3 interactions:  (100000, 3)


## yelp

In [8]:
df_yelp_inter = pd.read_csv(data_path_yelp + 'user_business.dat', sep='\t', header=None).rename({
    0: 'user_id',
    1: 'item_id',
    2: 'rating',
}, axis=1)
df_yelp_inter = df_yelp_inter[['user_id', 'item_id', 'rating']]
df_yelp_inter['user_id'] = df_yelp_inter['user_id'].astype('str')
df_yelp_inter['item_id'] = df_yelp_inter['item_id'].astype('str')
df_yelp_inter, ls_user_yelp = filter_interaction(df_yelp_inter)

df_yelp_inter.to_csv(data_preprocessing + "yelp_inter.csv", index=False)

origin shape:  (198397, 3)
> 3 interactions:  (188456, 3)
